# Resampling Images

## Overview

### Define output image information

* When resampling an image, we must first define output the image information, i.e. the *Origin*, *Spacing*, *Direction*, and *Size*.
* This determines the grid of samples where the output will be sampled.
* It does need not need to be the same as the input image.

![Resampling output information](Data/ResampleOutputInformation.png)

### Finding a resampled pixel's value

For each pixel in the output image:

1. Transform its index value to its location in physical space.
2. Apply an optional spatial transformation.
3. Transform the physical point location to a continuous index on the image to be resampled.
4. Interpolate from neighboring pixels.

![Common reference system](Data/CommonReferenceSystem.png)

### Different interpolators

<a href="http://www.itk.org/SimpleITKDoxygen/html/namespaceitk_1_1simple.html#a7cb1ef8bd02c669c02ea2f9f5aa374e5">There are many interpolators available</a>:
<ul>
<li> sitkNearestNeighbor </li>
<li> sitkLinear </li>
<li> sitkBSpline </li>
<li> sitkGaussian </li>
<li> sitkHammingWindowedSinc </li>
<li> sitkCosineWindowedSinc </li>
<li> sitkWelchWindowedSinc </li>
<li> sitkLanczosWindowedSinc </li>
<li> sitkBlackmanWindowedSinc </li>
</ul>

### Resampling and the registration framework

**Registration** involves sampling the two images to be registered with at *spatial transform*. The sampled values are compared with a *similarity metric*.  An *optimizer* is used to improve the spatial transformation parameters iteratively until the samples coorespond.

![Registration framework](Data/ITKv4RegistrationComponentsDiagram.svg)

## Tutorial

## Exercises